## Proj. Asn. 03 : Colin Kirby

In [1]:
# Imports.
import glm
import numpy as np
import pygame
import moderngl
from math import cos, sin, sqrt
from LoadObject import getObjectData
import ctypes

pygame 2.6.0 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html
Normal exists
texture exists
Normal exists
texture exists
Normal computed.
Main Program Parameters
in_normal <class '_moderngl.Attribute'> <Attribute: 1>
in_position <class '_moderngl.Attribute'> <Attribute: 0>
in_uv <class '_moderngl.Attribute'> <Attribute: 2>
eye <class '_moderngl.Uniform'> <Uniform: 0>
light <class '_moderngl.Uniform'> <Uniform: 1>
lightIntensity <class '_moderngl.Uniform'> <Uniform: 2>
map <class '_moderngl.Uniform'> <Uniform: 3>
metal <class '_moderngl.Uniform'> <Uniform: 4>
model <class '_moderngl.Uniform'> <Uniform: 5>
normalMap <class '_moderngl.Uniform'> <Uniform: 6>
perspective <class '_moderngl.Uniform'> <Uniform: 7>
uv_scale <class '_moderngl.Uniform'> <Uniform: 8>
view <class '_moderngl.Uniform'> <Uniform: 9>
Light Program Parameters
in_normal <class '_moderngl.Attribute'> <Attribute: 1>
in_position <class '_moderngl.Attribute'> <Attribute: 0>
m

In [ ]:
# Ensure high-DPI scaling is handled correctly on Windows.
ctypes.windll.user32.SetProcessDPIAware()

In [ ]:
# Load Objects.
files = ["teapot_with_texCoords.obj", "cube.obj", "20_icosahedron.obj"]

# Load Obj Data w/ Normals & Textures.
teapot_object = getObjectData(files[0], normal=True, texture=True)
box_object = getObjectData(files[1], normal=True, texture=True)
light_object = getObjectData(files[2], normal=True)

In [ ]:
# Define Trans. Matrices for Each Obj.
box_object_transformation = glm.scale(glm.vec3(10, 0.05, 10)) * glm.translate(glm.vec3(0, -1, 0))
teapot_object_transformation = glm.scale(glm.vec3(0.3)) * glm.translate(glm.vec3(0, 7.875, 0))
light_object_scaletransformation = glm.scale(glm.vec3(0.1))

In [ ]:
# Initialize Pygame.
pygame.init()  
clock = pygame.time.Clock()
window = pygame.display.set_mode(
    (1000, 800),
    flags=pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE,
    vsync=True
)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MAJOR_VERSION, 4)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MINOR_VERSION, 1)
pygame.display.set_caption(title="Project Assignment 3: Christopher Bowerfind")

# Create ModernGL context & Enable Depth Testing.
gl = moderngl.get_context()
gl.enable(gl.DEPTH_TEST)

# Get Initial Window Size.
width, height = window.get_size()
aspect_ratio = width / height


In [ ]:
# Load Image Func.
def load_image(image_file, channels="RGBA", flip_x=False, flip_y=False):
    texture_img = pygame.image.load(image_file)
    if flip_x or flip_y:
        texture_img = pygame.transform.flip(texture_img, flip_x, flip_y)
    texture_data = pygame.image.tobytes(texture_img, channels, True)
    return texture_data, texture_img.get_size()

In [ ]:
# Load Diffuse Texture for Teapot w/ Gold.
img_data, img_size = load_image("gold.jpg", "RGB")
_texture = gl.texture(img_size, 3, img_data)
_texture.build_mipmaps()
specular_sampler = gl.sampler(texture=_texture, filter=(gl.LINEAR_MIPMAP_LINEAR, gl.LINEAR))

# Load Diffuse Texture for Floor w/ Grass.
img_data, img_size = load_image('grass.jpg', "RGB")
_texture = gl.texture(img_size, 3, img_data)
_texture.build_mipmaps()
diffuse_sampler = gl.sampler(texture=_texture, filter=(gl.LINEAR_MIPMAP_LINEAR, gl.LINEAR))

# Load Normap Map for Teapot.
img_data, img_size = load_image("brick.png", "RGB")
_teapot_normal_texture = gl.texture(img_size, 3, img_data)
_teapot_normal_texture.build_mipmaps()
teapot_normal_sampler = gl.sampler(texture=_teapot_normal_texture, filter=(gl.LINEAR_MIPMAP_LINEAR, gl.LINEAR))

# Load Normal Map for Floor.
img_data, img_size = load_image("floorNormal.png", "RGB")
_floor_normal_texture = gl.texture(img_size, 3, img_data)
_floor_normal_texture.build_mipmaps()
floor_normal_sampler = gl.sampler(texture=_floor_normal_texture, filter=(gl.LINEAR_MIPMAP_LINEAR, gl.LINEAR))

In [ ]:
# Create Vertex Buffers for Teapot, Box, & Light.
teapot_vertex_buffer = gl.buffer(teapot_object[0])
box_object_buffer = gl.buffer(box_object[0])
light_object_buffer = gl.buffer(light_object[0])

In [ ]:
# Print Out Parameters of Shader Program.
def queryProgramParameters(program):
    for name in program:
        member = program[name]
        print(name, type(member), member)

In [ ]:
# Vertex Shader Code.
shared_vertex_shader_code = '''

#version 330 core

layout (location = 0) in vec4 in_position;
layout (location = 1) in vec3 in_normal;
layout (location = 2) in vec2 in_uv;

uniform mat4 model;
uniform mat4 view, perspective;
uniform vec2 uv_scale;  // Added uniform for UV scaling

out vec2 f_uv;
out vec3 f_normal;
out vec3 f_position;

void main() {
    f_position = (model * in_position).xyz;
    f_normal = normalize(mat3(transpose(inverse(model))) * in_normal);
    f_uv = in_uv * uv_scale;  // Apply UV scaling here
    gl_Position = perspective * view * vec4(f_position, 1.0);
}
'''


In [ ]:
# Main Program Parameters.
mainShaderProgram = gl.program(
    vertex_shader=shared_vertex_shader_code,
    fragment_shader='''
    
    #version 330 core

    in vec3 f_position;
    in vec3 f_normal;
    in vec2 f_uv;
    
    uniform vec3 eye;
    uniform sampler2D map;
    uniform sampler2D normalMap;  // Added uniform sampler2D for normal map
    uniform vec4 light;
    
    uniform bool metal;
    uniform float lightIntensity;  // Added uniform for light intensity
    const float shininess = 7.5;
    
    layout (location = 0) out vec4 out_color;
    
    // Function to compute the perturbed normal
    vec3 perturbNormal() {
        vec3 N = normalize(f_normal);
        vec3 dp1 = dFdx(f_position);
        vec3 dp2 = dFdy(f_position);
        vec2 duv1 = dFdx(f_uv);
        vec2 duv2 = dFdy(f_uv);
    
        vec3 T = normalize(duv2.t * dp1 - duv1.t * dp2);
        vec3 B = normalize(-duv2.s * dp1 + duv1.s * dp2);
    
        mat3 TBN = mat3(T, B, N);
    
        vec3 normalMapValue = texture(normalMap, f_uv).rgb;
        normalMapValue = normalMapValue * 2.0 - 1.0;  // Convert from [0,1] to [-1,1]
    
        vec3 N_p = normalize(TBN * normalMapValue);
        return N_p;
    }
    
    void main() {
        vec3 N = perturbNormal();  // Use the perturbed normal
        vec3 L = normalize(light.xyz);
        if (light.w > 0.0) L = normalize(light.xyz - f_position);
    
        vec3 V = normalize(eye - f_position);
        vec3 H = normalize(L + V);
    
        vec3 baseColor = texture(map, f_uv).rgb;
    
        vec3 color = ((metal) ? pow(clamp(dot(H, N), 0.0, 1.0), shininess) : clamp(dot(N, L), 0.0, 1.0)) * baseColor;
        color *= lightIntensity;  // Apply light intensity
        out_color = vec4(color, 1.0);
    }
'''
)
# Query and print shader program parameters for debugging
queryProgramParameters(mainShaderProgram)


In [ ]:
# Create Vertex Arrays for Teapots & Box.
teapot_renderable = gl.vertex_array(mainShaderProgram, [
    (teapot_vertex_buffer, '3f 3f 2f', 'in_position', 'in_normal', 'in_uv')
])

box_renderable = gl.vertex_array(mainShaderProgram, [
    (box_object_buffer, '3f 3f 2f', 'in_position', 'in_normal', 'in_uv')
])


In [ ]:
# Create a Separate Shader Program for Rendering Light Source.
lightShaderProgram = gl.program(
    vertex_shader=shared_vertex_shader_code,
    fragment_shader='''#version 330 core

in vec3 f_position;
in vec3 f_normal;

layout (location = 0) out vec4 out_color;

void main() {
    vec3 N = normalize(f_normal);
    out_color = vec4(0.5 * (N + 1.0), 1.0);
}'''
)
print("Light Program Parameters")
# Query and print shader program parameters for debugging
queryProgramParameters(lightShaderProgram)

# Create a vertex array for the light object using the light shader program
light_renderable = gl.vertex_array(lightShaderProgram, [
    (light_object_buffer, '3f 3f', 'in_position', 'in_normal')
])


In [ ]:
# Camera Config.
cameraDistance = 20
lookAtPoint = glm.vec3(0)
upVector = glm.vec3(0.0, 1.0, 0.0)
cameraStartDirection = glm.normalize(glm.vec3(0, 10, 20))
cameraOrbitAxis = glm.vec3(0, 1, 0)

def get_camera_matrix(angle):
    # Rotate the camera direction around the orbit axis
    viewDirection = glm.rotate(glm.radians(angle), cameraOrbitAxis) * cameraStartDirection
    eyePoint = lookAtPoint + cameraDistance * viewDirection
    viewMatrix = glm.lookAt(eyePoint, lookAtPoint, upVector)
    return viewMatrix, eyePoint

# Perspective Proj Parameters.
near = 1
far = 45
fov = 60

def getPerspectiveMatrix(aspect_ratio):
    return glm.perspective(glm.radians(fov), aspect_ratio, near, far)


In [ ]:
# Light Config.
lightDistance = 12
lightStartDirection = glm.normalize(glm.vec3(1, 1, 0))
lightTarget = glm.vec3(0)
lightOrbitAxis = glm.vec3(0, 1, 0)

def getLightVector(angle):
    # Rotate the light direction around the orbit axis
    lightVector = glm.rotate(glm.radians(angle), lightOrbitAxis) * lightStartDirection
    lightPosition = lightTarget + lightDistance * lightVector
    return lightVector, lightPosition


In [ ]:
# Func for Drawing Light.
def drawLight(lightPosition, view, perspective):
    program = lightShaderProgram
    program['view'].write(view)
    program["perspective"].write(perspective)
    
    # Apply Trans to Light Obj.
    light_model_transformation = glm.translate(lightPosition) * light_object_scaletransformation
    program["model"].write(light_model_transformation)
    
    # Render Light Obj.
    light_renderable.render()

# Func for Drawing Floor & Teapot.
def drawScene(light, eye, view, perspective):
    program = mainShaderProgram
    program["eye"].write(eye)
    program["light"].write(light)
    program['view'].write(view)
    program["perspective"].write(perspective)

    # Draw the floor
    program["metal"] = False
    program["lightIntensity"] = 1.0
    diffuse_sampler.use(0)
    program["map"] = 0
    floor_normal_sampler.use(1)
    program["normalMap"] = 1
    program["uv_scale"] = (2.0, 2.0)
    program["model"].write(box_object_transformation)
    box_renderable.render()

    # Draw the teapot
    program["metal"] = True
    program["lightIntensity"] = 2.0
    specular_sampler.use(0)
    program["map"] = 0
    teapot_normal_sampler.use(1)
    program["normalMap"] = 1
    program["uv_scale"] = (1.0, 1.0)
    program["model"].write(teapot_object_transformation)
    teapot_renderable.render()


In [ ]:
# Toggles for Controlling Animation & Light Source.
pause = True
pointSourceFlag = False

# Initialize Rotation Angle.
angle = 0
lightVector, lightPosition = getLightVector(angle)

# Calculate Initial Perspective Matrix.
perspectiveMatrix = getPerspectiveMatrix(aspect_ratio)

# Initialize Camera Once.
viewMatrix, eyePoint = get_camera_matrix(0)

In [ ]:
# Rendering Loop.
running = True

while running:
    clock.tick(60)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif (event.type == pygame.KEYDOWN):
            if event.key == pygame.K_ESCAPE:
                running = False
            elif event.key == pygame.K_p:
                pause = not pause  # Toggle pause
            elif event.key == pygame.K_l:
                pointSourceFlag = not pointSourceFlag
        elif (event.type == pygame.VIDEORESIZE):
            # Handle window resizing
            width, height = event.size
            aspect_ratio = width / height
            gl.viewport = (0, 0, width, height)
            perspectiveMatrix = getPerspectiveMatrix(aspect_ratio)

    gl.clear(0.0, 0.0, 0.0, depth=1.0)

    if not pause:
        angle += 1
        if angle > 360:
            angle -= 360

    lightVector, lightPosition = getLightVector(angle)

    drawLight(lightPosition, viewMatrix, perspectiveMatrix)

    light = glm.vec4(lightPosition, 1.0) if pointSourceFlag else glm.vec4(lightVector, 0.0)

    drawScene(light, eyePoint, viewMatrix, perspectiveMatrix)

    pygame.display.flip()

# Clean up and close Pygame when the loop ends
pygame.quit()
